# Data Cleaning

In [1]:
import csv
import os

In [ ]:
new_row = []
fieldNames = ['index','review']
with open('tokopedia_reviews_cleaned.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        new_row.append(row[1].replace('\n', ''))
index = len(new_row)
# Check entry number 2131, if it was not make sense, remove it
# Can also be used to remove any entry that is not make sense
# index_need_to_removed = [2131, 2183, 2183]
# index_need_to_removed = [2213]
# for i in index_need_to_removed:
#     print('Review at index',i,':',new_row[i])
#     new_row.pop(i)
#     print('Review at index',i,'(after):',new_row[i])
#     index = len(new_row)
#     print(len(new_row))


# with open('tokopedia_reviews_cleaned.csv', 'w') as file:
#     if not os.path.exists('tokopedia_reviews_cleaned.csv'):
#         writer = csv.DictWriter(file, fieldnames=fieldNames)
#         writer.writeheader()

#     for i in range (0, index):
#         csv_writer = csv.writer(file)
#         csv_writer.writerow([i,new_row[i]])

Review at index 2213 : yesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyes
Review at index 2213 (after): maantap, udah 2 kali beli disini
3226


# Data Labeling

In [25]:
import google.generativeai as genai
import time
genai.configure(api_key=os.environ.get('GENAI_API_KEY'))
gen_config = genai.types.GenerationConfig(temperature=0.0, max_output_tokens=100)

In [26]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [ ]:
def write_to_csv(filename:str, data:any, fieldnames:list[str]):
    """
    Write data to a CSV file, adding header only if the file doesn't exist.
    
    :param filename: Path to the CSV file
    :param data: List of dictionaries containing row data
    :param fieldnames: List of column names for the CSV header
    """
    # Check if the file exists
    file_exists = os.path.isfile(filename)
    
    # Open the file in append mode
    with open(filename, 'a', newline='') as csvfile:
        # Create a CSV writer object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write header only if the file doesn't exist
        if not file_exists:
            writer.writeheader()
        
        # Write the data rows
        for i, sentiment in enumerate(data):
            print(i, new_row[i], sentiment)
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow([i,new_row[i], sentiment])

In [28]:
def get_gemini_sentiment(text:str):
    """
    Get sentiment for a given text using the Gemini model.
    
    :param text: Text for which sentiment is to be determined
    :return: Sentiment of the text
    """
    # Generate sentiment using the Gemini model
    prompt = f"""
    Your role: sentiment analyzer
    Task: Answer the one of sentiment in the sentiment list (["dissapointed", "neutral", "satisfied"]) of each given message. Please only answer with the sentiment, for example if there is 3 messages, output as ['sentiment1', 'sentiment2', 'sentiment3'].
    Message: {text}
    """
    response = model.generate_content(prompt, generation_config=gen_config)
    sentiment = eval(response.text.strip())
    
    # Return the sentiment
    return sentiment

In [32]:
def write_to_csv(filename, data, fieldnames):
    """
    Write data to a CSV file, adding header only if the file doesn't exist.
    
    :param filename: Path to the CSV file
    :param data: List or single dictionary containing row data
    :param fieldnames: List of column names for the CSV header
    """
    # Ensure data is a list of dictionaries
    if not isinstance(data, list):
        data = [data]
    
    # Check if the file exists
    file_exists = os.path.isfile(filename)
    
    # Open the file in append mode
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        # Create a CSV writer object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write header only if the file doesn't exist
        if not file_exists:
            writer.writeheader()
        
        # Write the data rows
        writer.writerows(data)

def process_reviews_with_sentiment(new_row:any, start_index:int = 0, batch_size=6):
    """
    Process reviews in batches, get sentiment, and write to CSV
    
    :param new_row: List of reviews to process
    :param batch_size: Number of reviews to process in each batch
    """
    # Define CSV file and field names
    csv_filename = 'tokopedia_reviews_cleaned_sentiment.csv'
    new_fieldNames = ['original_index', 'review', 'sentiment']
    
    # Track current row
    current_row = start_index
    
    while current_row < len(new_row):
        try:
            # Determine the end of the current batch
            end_row = min(current_row + batch_size, len(new_row))
            
            # Create message with numbered reviews
            batch_reviews = new_row[current_row:end_row]
            message = ' '.join(f"{i+1}. {review}" for i, review in enumerate(batch_reviews))
            
            print(f'Processing batch from row {current_row} to {end_row}')
            print('Message:', message)
            
            # Get sentiment for the batch
            batch_sentiments = get_gemini_sentiment(message)
            print('Sentiments:', batch_sentiments)
            
            # Prepare data for CSV
            csv_data = [
                {
                    'original_index': current_row + i + 1,
                    'review': batch_reviews[i],
                    'sentiment': sentiment
                } 
                for i, sentiment in enumerate(batch_sentiments)
            ]
            
            # Write to CSV
            write_to_csv(csv_filename, csv_data, new_fieldNames)
            
            # Update current row
            current_row = end_row
            
            # Pause to avoid rate limiting
            time.sleep(2)
            
        except Exception as e:
            print(f"Error processing batch starting at row {current_row}: {e}")
            # Optional: break or continue based on your error handling strategy
            break

In [34]:
process_reviews_with_sentiment(new_row, start_index=0, batch_size=6)

Processing batch from row 0 to 6
Message: 1. Pelayanan ramah, baguss barangnya puaass 2. JUARA 3. pengemasan sangat rapih dan tebal respon CS cepat.Alhamdulillah tidak mengecewakan sudah beli beberapa produk dari doss 4. top dosss, performa oke, packing oke 5. selalu puas dan tenang belanja di doss mantap 6. Kualitas barang bagus, harga sangat kompetitif. pengiriman cepat dan aman. tim CS responsif. Semoga barang awet
Sentiments: ['satisfied', 'satisfied', 'satisfied', 'satisfied', 'satisfied', 'satisfied']
Processing batch from row 6 to 12
Message: 1. Terima kasih kak, keren ini inovasinya, semoga awet 2. Barang bagus dan sesuai deskripsi berfungsi dengan baik 3. belum kepasang jatoh dan ilang wkwk 4. mendarat dengan selamat, packingnya top buat pengiriman barang elektronik 5. Sesuai deskripsi, seller responsif 6. pelayanan cepat..barang kualitas bagus
Sentiments: ['satisfied', 'satisfied', 'dissapointed', 'satisfied', 'satisfied', 'satisfied']
Processing batch from row 12 to 18
Messa

In [ ]:
current_row = 0
new_fieldNames = ['index', 'review', 'sentiment']
while current_row < len(new_row):
    if current_row + 6 > len(new_row):
        message = ' '.join(f"{i+1}. {new_row[i]}" for i in range(current_row, len(new_row)))
        print('Message:', message)
        sentiment = get_gemini_sentiment(message)
        print('Sentiment:', sentiment)
        write_to_csv('tokopedia_reviews_cleaned_sentiment.csv', sentiment, new_fieldNames)
        time.sleep(2)
        break
    message = ' '.join(f"{i+1}. {new_row[i]}" for i in range(current_row, current_row+6))
    print('Message:',message)
    sentiment = get_gemini_sentiment(message)
    print('Sentiment:', sentiment)
    write_to_csv('tokopedia_reviews_cleaned_sentiment.csv', sentiment, new_fieldNames)    # response = genai.complete(message, config=gen_config)
    # new_row.insert(current_row+6, response.choices[0].text)
    current_row += 7
    time.sleep(2)

In [ ]:
message = ' '.join(f"{i+1}. {new_row[i]}" for i in range(0, 6))
prompt = f"""
    Your role: sentiment analyzer
    Task: Answer the one of sentiment in the sentiment list (["dissapointed", "neutral", "satisfied"]) of each given message. Please only answer with the sentiment, for example if there is 3 messages, output as ['sentiment1', 'sentiment2', 'sentiment3'].
    Message: {message}
    """

In [9]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
response = model.generate_content(prompt, generation_config=gen_config)
print(response.text)

['satisfaction', 'satisfaction', 'satisfaction', 'satisfaction', 'satisfaction', 'satisfaction']



In [ ]:
# Assuming response.text is a list of sentiments
data = eval(response.text.strip())
print(data[0])
new_fieldNames = ['index', 'review', 'sentiment']

write_to_csv('test.csv', data, new_fieldNames)


satisfaction
0 Pelayanan ramah, baguss barangnya puaass satisfaction
1 JUARA satisfaction
2 pengemasan sangat rapih dan tebal respon CS cepat.Alhamdulillah tidak mengecewakan sudah beli beberapa produk dari doss satisfaction
3 top dosss, performa oke, packing oke satisfaction
4 selalu puas dan tenang belanja di doss mantap satisfaction
5 Kualitas barang bagus, harga sangat kompetitif. pengiriman cepat dan aman. tim CS responsif. Semoga barang awet satisfaction


# Modeling

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

2024-11-28 17:49:19.530883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 17:49:19.530930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 17:49:19.531527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 17:49:19.535114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 17:49:20.068542: W tensorflow/compiler/tf2

In [2]:
dataset = pd.read_csv('tokopedia_reviews_cleaned_sentiment.csv')
dataset.tail()

,original_index,review,sentiment
3245,3251,"Kualitas barang bagus, harga bersaing. Pengiri...",satisfied
3246,3252,Terang dari flash nya oke biasa saja. Packagin...,neutral
3247,3253,Kabel XLR berfungsi normal. Packaging rapi. Pe...,neutral
3248,3254,Kit is complete. Everything seems to work fine...,satisfied
3249,3255,Paket datang dalam kondisi oke.Pengiriman agak...,neutral


In [3]:
dataset['sentiment'].unique()

array(['satisfied', 'dissapointed', 'neutral'], dtype=object)

In [3]:
vocab_size = 10000
embedding_dim = 16
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 2000

sentences = []
labels = []

for item in dataset['review']:
    sentences.append(item)
for item in dataset['sentiment']:
    if item == 'dissapointed':
        labels.append(0)
    elif item == 'neutral':
        labels.append(1)
    elif item == 'satisfied':
        labels.append(2)
    # labels.append(item)

In [4]:
training_sentences, testing_sentences, training_label, testing_label = train_test_split(sentences, labels, test_size=0.2, random_state=69)

In [ ]:
# Fit your tokenizer with training data
# tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)  # YOUR CODE HERE
# tokenizer.fit_on_texts(sentences)
# training_sentence=tokenizer.texts_to_sequences(sentences[:training_size])
# training_padded=pad_sequences(training_sentence,maxlen=max_length,truncating=trunc_type,padding=padding_type,value=0)

# testing_sentence=tokenizer.texts_to_sequences(sentences[training_size:])
# testing_padded=pad_sequences(testing_sentence,maxlen=max_length,truncating=trunc_type,padding=padding_type,value=0)

# training_labels=np.array(labels[:training_size])
# testing_labels=np.array(labels[training_size:])

In [5]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)  # YOUR CODE HERE
tokenizer.fit_on_texts(sentences)
training_sentence=tokenizer.texts_to_sequences(training_sentences)
training_padded=pad_sequences(training_sentence,maxlen=max_length,truncating=trunc_type,padding=padding_type,value=0)

testing_sentence=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sentence,maxlen=max_length,truncating=trunc_type,padding=padding_type,value=0)

training_labels=np.array(training_label)
testing_labels=np.array(testing_label)

In [6]:
print(training_padded.shape)
print(testing_padded.shape)

(2600, 200)
(650, 200)


In [7]:
print(training_labels.shape)
print(testing_labels.shape)

(2600,)
(650,)


In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(24, activation='relu'),
        # YOUR CODE HERE. DO not change the last layer or test may fail
        tf.keras.layers.Dense(3, activation='softmax')
    ])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(training_padded,training_labels,epochs=10,validation_data=(testing_padded,testing_labels))   


Epoch 1/10
82/82 [==============================] - 18s 141ms/step - loss: 0.5306 - accuracy: 0.8731 - val_loss: 0.4285 - val_accuracy: 0.8846
Epoch 2/10
82/82 [==============================] - 8s 101ms/step - loss: 0.3459 - accuracy: 0.8912 - val_loss: 0.3205 - val_accuracy: 0.8954
Epoch 3/10
82/82 [==============================] - 7s 86ms/step - loss: 0.1971 - accuracy: 0.9288 - val_loss: 0.4434 - val_accuracy: 0.8908
Epoch 4/10
82/82 [==============================] - 10s 122ms/step - loss: 0.1460 - accuracy: 0.9427 - val_loss: 0.3905 - val_accuracy: 0.9000
Epoch 5/10
82/82 [==============================] - 7s 86ms/step - loss: 0.1075 - accuracy: 0.9612 - val_loss: 0.4834 - val_accuracy: 0.8892
Epoch 6/10
82/82 [==============================] - 8s 93ms/step - loss: 0.0847 - accuracy: 0.9700 - val_loss: 0.5180 - val_accuracy: 0.8846
Epoch 7/10
82/82 [==============================] - 7s 89ms/step - loss: 0.0678 - accuracy: 0.9785 - val_loss: 0.5965 - val_accuracy: 0.9000
Epoch 8/